# Extending Standard Algorithms for Particle Simulations

In this notebook, we'll explore techniques for extending standard algorithms to solve problems efficiently in our particle simulation system. We'll look at how to use iterators to optimize memory usage and improve performance.


<img src="./images/image4.svg" width="800" height="600">




## Table of Contents

- The Challenge: Finding Maximum Displacement
- Naive Implementation
- Understanding Iterators
- CUDA Fancy Iterators
- Optimized Implementation
- Performance Evaluation

## The Challenge: Finding Maximum Displacement

In our particle simulation, we want to track how much particles move between timesteps. Specifically, we want to find the maximum displacement of any particle in each simulation step.

Given two snapshots of our particle system (before and after a simulation step), we need to compute the maximum displacement across all particles.

Naive Implementation
Let's start with a straightforward implementation that computes the displacements and then finds the maximum:

In [2]:
#Specifying path to where nvcc exists so that the jupyter notebook reads from it. nvcc is the nvidia cuda compiler for executing cuda. 
import os
os.environ['PATH'] = "/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.6.1-cf4xlcbcfpwchqwo5bktxyhjagryzcx6/bin:" + os.environ['PATH']

In [3]:
%%writefile codes/naive_max_displacement.cu
#include <thrust/universal_vector.h>
#include <thrust/transform.h>
#include <thrust/execution_policy.h>
#include <thrust/reduce.h>
#include <cstdio>
#include <cmath>

struct Particle {
    float x, y;    // position
    float vx, vy;  // velocity
};

// Calculate distance between two positions
float distance(float x1, float y1, float x2, float y2) {
    float dx = x2 - x1;
    float dy = y2 - y1;
    return sqrt(dx*dx + dy*dy);
}

// Naive approach to find maximum displacement
float naive_max_displacement(const thrust::universal_vector<Particle>& before, 
                           const thrust::universal_vector<Particle>& after) 
{
    // Allocate vector to store displacements (temporary storage)
    thrust::universal_vector<float> displacements(before.size());

    // Compute displacements
    thrust::transform(thrust::device, 
                     before.begin(), before.end(),           // first input sequence  
                     after.begin(),                          // second input sequence
                     displacements.begin(),                  // output
                     [=] __host__ __device__ (const Particle& p1, const Particle& p2) {
                         // Calculate displacement between positions
                         return distance(p1.x, p1.y, p2.x, p2.y);
                     });

    // Find maximum displacement
    return thrust::reduce(thrust::device, 
                         displacements.begin(), 
                         displacements.end(), 
                         0.0f, thrust::maximum<float>{});
}

int main() 
{
    float dt = 0.1f;  // time step
    
    // Create two arrays to store particle states in alternating steps
    thrust::universal_vector<Particle> particles[2] = {
        // Initial state
        {
            {0.0f, 0.0f, 1.0f, 0.5f},    // Particle 1
            {1.0f, 2.0f, -0.5f, 0.2f},   // Particle 2
            {-1.0f, -1.0f, 0.3f, 0.7f}   // Particle 3
        },
        // Empty state for next timestep
        {}
    };
    particles[1].resize(particles[0].size());
    
    // Update function - applies velocity to position
    auto update_particle = [=] __host__ __device__ (const Particle& p) {
        Particle updated = p;
        updated.x += p.vx * dt;
        updated.y += p.vy * dt;
        return updated;
    };
    
    std::printf("step  max_displacement\n");
    for (int step = 0; step < 3; step++) {
        // Get references to current and next state arrays
        thrust::universal_vector<Particle>& current = particles[step % 2];
        thrust::universal_vector<Particle>& next = particles[(step + 1) % 2];
        
        // Update particle positions
        thrust::transform(thrust::device, 
                         current.begin(), current.end(), 
                         next.begin(), 
                         update_particle);
        
        // Calculate maximum displacement
        float max_disp = naive_max_displacement(current, next);
        std::printf("%d     %.4f\n", step, max_disp);
    }
    
    return 0;
}

Overwriting codes/naive_max_displacement.cu


Let's compile and run this:


In [4]:
%%bash
nvcc --extended-lambda -o codes/naive_displacement codes/naive_max_displacement.cu
./codes/naive_displacement

codes/naive_max_displacement.cu(34): warning #20011-D: calling a __host__ function("distance(float, float, float, float)") from a __host__ __device__ function("naive_max_displacement(const  ::thrust::THRUST_200500_520_NS::detail::vector_base<    ::Particle,  ::thrust::THRUST_200500_520_NS::mr::stateless_resource_allocator<    ::Particle,  ::thrust::THRUST_200500_520_NS::system::cuda::detail::cuda_memory_resource<&thrust::THRUST_200500_520_NS::system::cuda::detail::cudaMallocManaged, &::cudaFree,  ::thrust::THRUST_200500_520_NS::pointer<void,  ::thrust::THRUST_200500_520_NS::cuda_cub::tag, void,  ::thrust::THRUST_200500_520_NS::use_default> > > >  &, const  ::thrust::THRUST_200500_520_NS::detail::vector_base<    ::Particle,  ::thrust::THRUST_200500_520_NS::mr::stateless_resource_allocator<    ::Particle,  ::thrust::THRUST_200500_520_NS::system::cuda::detail::cuda_memory_resource<&thrust::THRUST_200500_520_NS::system::cuda::detail::cudaMallocManaged, &::cudaFree,  ::thrust::THRUST_200500

step  max_displacement
0     0.0000
1     0.0000
2     0.0000


This implementation works but has a key inefficiency: **we allocate temporary storage** for all the displacements, even though we only need the maximum value. For large simulations, this wastes valuable GPU memory and requires extra memory operations.

Let's count the memory operations:

- Read positions from before vector: 2n floats (x, y for each particle)
- Read positions from after vector: 2n floats
- Write displacements to temporary storage: n floats
- Read displacements for reduction: n floats

Total: 5n memory operations. If we could avoid the temporary storage, we could reduce this to 4n operations.

## Understanding Iterators

Before optimizing our code, let's understand what iterators are and how they can help us.

Iterators can be thought of as a generalization of pointers:

- They point to elements in a sequence
- They can be dereferenced to access the current element
- They can be incremented to point to the next element

The key difference is that iterators can perform computations on-the-fly when dereferenced, without storing intermediate results.

Let's look at a few simple iterator examples:

## Simple Counting Iterator


In [13]:
%%writefile codes/counting_iterator.cu
#include <cstdio>

// A simple iterator that generates a sequence of integers
struct CountingIterator {
    // Returns the value at index i
    __host__ __device__
    int operator[](int i) {
        return i;
    }
};

int main() {
    CountingIterator counter;
    
    printf("counter[0]: %d\n", counter[0]);  // Prints 0
    printf("counter[5]: %d\n", counter[5]);  // Prints 5
    printf("counter[10]: %d\n", counter[10]);  // Prints 10
    
    return 0;
}

Overwriting codes/counting_iterator.cu


In [16]:
%%bash
nvcc -o codes/counting_iterator codes/counting_iterator.cu
./codes/counting_iterator


counter[0]: 0
counter[5]: 5
counter[10]: 10


This iterator generates an infinite sequence of integers without allocating any storage!


## Transform Iterator
Let's create an iterator that transforms values from an underlying container:


In [17]:
%%writefile codes/transform_iterator.cu
#include <cstdio>
#include <array>

// An iterator that transforms values from an array
struct TransformIterator {
    float* data;
    
    // Multiplies each value by 2
    __host__ __device__
    float operator[](int i) {
        return data[i] * 2.0f;
    }
};

int main() {
    std::array<float, 3> values{1.0f, 2.0f, 3.0f};
    
    TransformIterator transformer{values.data()};
    
    printf("Original: %.1f, Transformed: %.1f\n", values[0], transformer[0]);  // 1.0, 2.0
    printf("Original: %.1f, Transformed: %.1f\n", values[1], transformer[1]);  // 2.0, 4.0
    printf("Original: %.1f, Transformed: %.1f\n", values[2], transformer[2]);  // 3.0, 6.0
    
    return 0;
}

Writing codes/transform_iterator.cu


In [18]:
%%bash
nvcc -o codes/transform codes/transform_iterator.cu
./codes/transform

Original: 1.0, Transformed: 2.0
Original: 2.0, Transformed: 4.0
Original: 3.0, Transformed: 6.0


## Zip Iterator

Now let's create an iterator that combines elements from two containers:

In [19]:
%%writefile codes/zip_iterator.cu
#include <cstdio>
#include <array>
#include <tuple>

// An iterator that combines elements from two arrays
struct ZipIterator {
    float* positions;
    float* velocities;
    
    // Returns a tuple containing the position and velocity
    __host__ __device__
    std::tuple<float, float> operator[](int i) {
        return {positions[i], velocities[i]};
    }
};

int main() {
    std::array<float, 3> positions{0.0f, 1.0f, 2.0f};
    std::array<float, 3> velocities{0.5f, -0.3f, 1.0f};
    
    ZipIterator zipIter{positions.data(), velocities.data()};
    
    for (int i = 0; i < 3; i++) {
        auto [pos, vel] = zipIter[i];
        printf("Particle %d: position=%.1f, velocity=%.1f\n", i, pos, vel);
    }
    
    return 0;
}

Writing codes/zip_iterator.cu


In [20]:
%%bash
nvcc -o codes/zip codes/zip_iterator.cu
./codes/zip

codes/zip_iterator.cu(24): warning #3356-D: structured bindings are a C++17 feature
          auto [pos, vel] = zipIter[i];
               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

codes/zip_iterator.cu(13): warning #20015-D: calling a constexpr __host__ function from a __host__ __device__ function is not allowed. The experimental flag '--expt-relaxed-constexpr' can be used to allow this.
          return {positions[i], velocities[i]};
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

codes/zip_iterator.cu(24): warning #3356-D: structured bindings are a C++17 feature
          auto [pos, vel] = zipIter[i];
               ^

codes/zip_iterator.cu(13): warning #20013-D: calling a constexpr __host__ function("std::tuple<float, float > ::tuple<float &, float &, (bool)1> (T1 &&, T2 &&)") from a __host__ __device__ function("ZipIterator::operator []") is not allowed. The experimental flag '--expt-relaxed-conste

Particle 0: position=0.0, velocity=0.5
Particle 1: position=1.0, velocity=-0.3
Particle 2: position=2.0, velocity=1.0


## Combining Iterators

The real power comes when we combine these iterator concepts. Let's create a combined iterator that can compute particle displacements without temporary storage:

In [20]:
%%writefile codes/combined_iterator.cu
#include <cstdio>
#include <array>
#include <tuple>
#include <cmath>

struct Particle {
    float x, y;    // position
};

// Zip iterator to combine two particle arrays
struct ParticleZipIterator {
    Particle* before;
    Particle* after;
    
    // Returns a tuple containing both particle states
    __host__ __device__
    std::tuple<Particle, Particle> operator[](int i) {
        return {before[i], after[i]};
    }
};

// Displacement iterator that computes distance between particle positions
struct DisplacementIterator {
    ParticleZipIterator zip;
    
    // Computes distance without storing intermediate results
    __host__ __device__
    float operator[](int i) {
        auto [p1, p2] = zip[i];
        float dx = p2.x - p1.x;
        float dy = p2.y - p1.y;
        return sqrt(dx*dx + dy*dy);
    }
};

int main() {
    Particle before[3] = {
        {0.0f, 0.0f},
        {1.0f, 2.0f},
        {-1.0f, -1.0f}
    };
    
    Particle after[3] = {
        {0.1f, 0.05f},
        {0.95f, 2.02f},
        {-0.97f, -0.93f}
    };
    
    ParticleZipIterator zipIter{before, after};
    DisplacementIterator dispIter{zipIter};
    
    for (int i = 0; i < 3; i++) {
        printf("Particle %d displacement: %.4f\n", i, dispIter[i]);
    }
    
    // Find maximum displacement manually
    float max_disp = 0.0f;
    for (int i = 0; i < 3; i++) {
        max_disp = fmax(max_disp, dispIter[i]);
    }
    
    printf("Maximum displacement: %.4f\n", max_disp);
    
    return 0;
}

Overwriting codes/combined_iterator.cu


In [21]:
%%bash
nvcc -o codes/combined_iterator codes/combined_iterator.cu
./codes/combined_iterator

codes/combined_iterator.cu(29): warning #3356-D: structured bindings are a C++17 feature
          auto [p1, p2] = zip[i];
               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

codes/combined_iterator.cu(18): warning #20015-D: calling a constexpr __host__ function from a __host__ __device__ function is not allowed. The experimental flag '--expt-relaxed-constexpr' can be used to allow this.
          return {before[i], after[i]};
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

codes/combined_iterator.cu(29): warning #3356-D: structured bindings are a C++17 feature
          auto [p1, p2] = zip[i];
               ^

codes/combined_iterator.cu(29): warning #20015-D: calling a constexpr __host__ function from a __host__ __device__ function is not allowed. The experimental flag '--expt-relaxed-constexpr' can be used to allow this.
          auto [p1, p2] = zip[i];
                ^

codes/combined_iterato

Particle 0 displacement: 0.1118
Particle 1 displacement: 0.0539
Particle 2 displacement: 0.0762
Maximum displacement: 0.1118


This example shows how we can compute displacements on-the-fly without storing intermediate results!

# CUDA Fancy Iterators

Thrust provides several built-in iterators that implement these concepts in a GPU-optimized way:

- thrust::counting_iterator: Generates a sequence of numbers
- thrust::transform_iterator: Applies a function to elements of another iterator
- thrust::zip_iterator: Combines multiple iterators into one
- thrust::permutation_iterator: Reorders elements of another iterator
- thrust::discard_iterator: Ignores any values assigned to it

Let's see how to use these in our particle simulation:

## Using Zip Iterator
First, let's see how to use the thrust::zip_iterator:

In [8]:
%%writefile codes/thrust_zip.cu
#include <thrust/device_vector.h>  // Use device_vector instead of universal_vector for compatibility
#include <thrust/iterator/zip_iterator.h>  // Correct path to zip_iterator
#include <thrust/tuple.h>  // Explicitly include tuple
#include <thrust/execution_policy.h>
#include <cstdio>

struct Particle {
    float x, y;    // position
};

int main() {
    // Create two sets of particles
    thrust::device_vector<Particle> before{
        {0.0f, 0.0f},
        {1.0f, 2.0f},
        {-1.0f, -1.0f}
    };
    
    thrust::device_vector<Particle> after{
        {0.1f, 0.05f},
        {0.95f, 2.02f},
        {-0.97f, -0.93f}
    };
    
    // Create a zip iterator to combine both vectors
    auto zip = thrust::make_zip_iterator(thrust::make_tuple(before.begin(), after.begin()));
    
    // Access the first particle pair
    auto pair = *zip;
    Particle p1 = thrust::get<0>(pair);  // Use copy instead of reference for device memory
    Particle p2 = thrust::get<1>(pair);  // Use copy instead of reference for device memory
    
    printf("First particle before: (%.2f, %.2f)\n", p1.x, p1.y);
    printf("First particle after: (%.2f, %.2f)\n", p2.x, p2.y);
    
    return 0;
}

Overwriting codes/thrust_zip.cu


In [9]:
%%bash
nvcc -o codes/thrust_zip codes/thrust_zip.cu
./codes/thrust_zip

First particle before: (0.00, 0.00)
First particle after: (0.10, 0.05)


## Combining Zip and Transform Iterators
Now let's combine **thrust::zip_iterator** with **thrust::transform_iterator** to compute displacements:

In [10]:
%%writefile codes/thrust_combined.cu
#include <thrust/device_vector.h>  // Use device_vector instead of universal_vector for better compatibility
#include <thrust/iterator/zip_iterator.h>  // Correct path for zip iterator
#include <thrust/iterator/transform_iterator.h>  // Correct path for transform iterator
#include <thrust/tuple.h>  // Explicitly include tuple header
#include <thrust/execution_policy.h>
#include <cstdio>
#include <cmath>

struct Particle {
    float x, y;    // position
};

int main() {
    // Create two sets of particles
    thrust::device_vector<Particle> before{
        {0.0f, 0.0f},
        {1.0f, 2.0f},
        {-1.0f, -1.0f}
    };
    
    thrust::device_vector<Particle> after{
        {0.1f, 0.05f},
        {0.95f, 2.02f},
        {-0.97f, -0.93f}
    };
    
    // Create a zip iterator
    auto zip = thrust::make_zip_iterator(thrust::make_tuple(before.begin(), after.begin()));
    
    // Create a transform iterator that computes displacement
    auto displacement = thrust::make_transform_iterator(zip, 
        [] __host__ __device__ (const thrust::tuple<Particle, Particle>& t) {
            const Particle& p1 = thrust::get<0>(t);
            const Particle& p2 = thrust::get<1>(t);
            
            float dx = p2.x - p1.x;
            float dy = p2.y - p1.y;
            return sqrt(dx*dx + dy*dy);
        });
    
    // Print the first few displacements
    printf("Particle 0 displacement: %.4f\n", displacement[0]);
    printf("Particle 1 displacement: %.4f\n", displacement[1]);
    printf("Particle 2 displacement: %.4f\n", displacement[2]);
    
    return 0;
}

Overwriting codes/thrust_combined.cu


In [11]:
%%bash
nvcc --extended-lambda -o codes/thrust_combined codes/thrust_combined.cu
./codes/thrust_combined 

Particle 0 displacement: 0.1118
Particle 1 displacement: 0.0539
Particle 2 displacement: 0.0762


## Optimized Implementation
Now we can rewrite our maximum displacement function using these iterators:

In [16]:
%%writefile codes/optimized_max_displacement.cu

#include <thrust/device_vector.h>  // Changed from universal_vector
#include <thrust/transform.h>
#include <thrust/execution_policy.h>
#include <thrust/iterator/zip_iterator.h>  // Fixed path
#include <thrust/iterator/transform_iterator.h>  // Fixed path
#include <thrust/reduce.h>
#include <thrust/tuple.h>  // Added missing header
#include <cstdio>
#include <cmath>

struct Particle {
    float x, y;    // position
    float vx, vy;  // velocity
};

// Optimized approach using iterators
float optimized_max_displacement(const thrust::device_vector<Particle>& before, 
                                const thrust::device_vector<Particle>& after) 
{
    // Create a zip iterator to combine both vectors
    auto zip = thrust::make_zip_iterator(thrust::make_tuple(before.begin(), after.begin()));
    
    // Create a transform iterator that computes displacement
    auto displacement = thrust::make_transform_iterator(zip, 
        [] __host__ __device__ (const thrust::tuple<Particle, Particle>& t) {  // Fixed attributes
            const Particle& p1 = thrust::get<0>(t);
            const Particle& p2 = thrust::get<1>(t);
            
            float dx = p2.x - p1.x;
            float dy = p2.y - p1.y;
            return sqrt(dx*dx + dy*dy);
        });
    
    // Find maximum displacement directly from the transform iterator
    return thrust::reduce(thrust::device, 
                         displacement, 
                         displacement + before.size(),
                         0.0f, thrust::maximum<float>{});
}

int main() 
{
    float dt = 0.1f;  // time step
    
    // Create two arrays to store particle states in alternating steps
    thrust::device_vector<Particle> particles[2] = {  // Changed from universal_vector
        // Initial state
        {
            {0.0f, 0.0f, 1.0f, 0.5f},    // Particle 1
            {1.0f, 2.0f, -0.5f, 0.2f},   // Particle 2
            {-1.0f, -1.0f, 0.3f, 0.7f}   // Particle 3
        },
        // Empty state for next timestep
        {}
    };
    particles[1].resize(particles[0].size());
    
    // Update function - applies velocity to position
    auto update_particle = [=] __host__ __device__ (const Particle& p) {  // Fixed attributes
        Particle updated = p;
        updated.x += p.vx * dt;
        updated.y += p.vy * dt;
        return updated;
    };
    
    std::printf("step  max_displacement\n");
    for (int step = 0; step < 3; step++) {
        // Get references to current and next state arrays
        thrust::device_vector<Particle>& current = particles[step % 2];
        thrust::device_vector<Particle>& next = particles[(step + 1) % 2];
        
        // Update particle positions
        thrust::transform(thrust::device, 
                         current.begin(), current.end(), 
                         next.begin(), 
                         update_particle);
        
        // Calculate maximum displacement
        float max_disp = optimized_max_displacement(current, next);
        std::printf("%d     %.4f\n", step, max_disp);
    }
    
return 0;
}

Overwriting codes/optimized_max_displacement.cu


In [17]:
%%bash
nvcc --extended-lambda -o codes/optimized_max_displacement codes/optimized_max_displacement.cu
./codes/optimized_max_displacement 

step  max_displacement
0     0.1118
1     0.1118
2     0.1118


The key improvement is that we no longer need to allocate a temporary vector to store displacements. The transform iterator computes them on-the-fly during the reduction operation.


# Performance Evaluation
Let's compare the performance of both approaches with a large number of particles:

In [18]:
%%writefile codes/performance_comparison.cu

#include <thrust/device_vector.h>  // Changed from universal_vector
#include <thrust/transform.h>
#include <thrust/execution_policy.h>
#include <thrust/iterator/zip_iterator.h>  // Fixed path
#include <thrust/iterator/transform_iterator.h>  // Fixed path
#include <thrust/reduce.h>
#include <thrust/sequence.h>
#include <thrust/tuple.h>  // Added missing header
#include <chrono>
#include <cstdio>
#include <cmath>

struct Particle {
    float x, y;    // position
};

// Naive approach with temporary storage
float naive_max_displacement(const thrust::device_vector<Particle>& before, 
                           const thrust::device_vector<Particle>& after) 
{
    // Allocate vector to store displacements
    thrust::device_vector<float> displacements(before.size());
    // Compute displacements
    thrust::transform(thrust::device, 
                     before.begin(), before.end(),
                     after.begin(),
                     displacements.begin(),
                     [] __host__ __device__ (const Particle& p1, const Particle& p2) {  // Fixed attributes
                         float dx = p2.x - p1.x;
                         float dy = p2.y - p1.y;
                         return sqrt(dx*dx + dy*dy);
                     });
    // Find maximum displacement
    return thrust::reduce(thrust::device, 
                         displacements.begin(), 
                         displacements.end(), 
                         0.0f, thrust::maximum<float>{});
}

// Optimized approach using iterators
float optimized_max_displacement(const thrust::device_vector<Particle>& before, 
                                const thrust::device_vector<Particle>& after) 
{
    // Create a zip iterator
    auto zip = thrust::make_zip_iterator(thrust::make_tuple(before.begin(), after.begin()));
    
    // Create a transform iterator
    auto displacement = thrust::make_transform_iterator(zip, 
        [] __host__ __device__ (const thrust::tuple<Particle, Particle>& t) {  // Fixed attributes
            const Particle& p1 = thrust::get<0>(t);
            const Particle& p2 = thrust::get<1>(t);
            
            float dx = p2.x - p1.x;
            float dy = p2.y - p1.y;
            return sqrt(dx*dx + dy*dy);
        });
    
    // Find maximum displacement
    return thrust::reduce(thrust::device, 
                         displacement, 
                         displacement + before.size(),
                         0.0f, thrust::maximum<float>{});
}

int main() 
{
    // Create large particle arrays (16 million particles)
    // Reduced size for testing - 16M particles may be too much for some systems
    const int NUM_PARTICLES = 1 << 20;  // Using 1M particles instead of 16M for initial testing
    
    thrust::device_vector<Particle> before(NUM_PARTICLES);
    thrust::device_vector<Particle> after(NUM_PARTICLES);
    
    // Initialize with some values
    // Note: This initialization is inefficient - should use thrust::transform for GPU
    thrust::device_vector<int> indices(NUM_PARTICLES);
    thrust::sequence(indices.begin(), indices.end());
    
    thrust::transform(indices.begin(), indices.end(), before.begin(),
        [] __host__ __device__ (int i) {
            Particle p;
            p.x = i;
            p.y = NUM_PARTICLES - i;
            return p;
        });
        
    thrust::transform(indices.begin(), indices.end(), after.begin(),
        [] __host__ __device__ (int i) {
            Particle p;
            p.x = i + 0.5f;
            p.y = NUM_PARTICLES - i - 0.2f;
            return p;
        });
    
    // Measure naive approach time (include allocation)
    auto start_naive = std::chrono::high_resolution_clock::now();
    float naive_result = naive_max_displacement(before, after);
    auto end_naive = std::chrono::high_resolution_clock::now();
    
    // Measure optimized approach time
    auto start_optimized = std::chrono::high_resolution_clock::now();
    float optimized_result = optimized_max_displacement(before, after);
    auto end_optimized = std::chrono::high_resolution_clock::now();
    
    // Calculate durations in milliseconds
    float naive_time = std::chrono::duration<float, std::milli>(end_naive - start_naive).count();
    float optimized_time = std::chrono::duration<float, std::milli>(end_optimized - start_optimized).count();
    
    // Print results
    printf("Naive approach: %.3f ms, result = %.6f\n", naive_time, naive_result);
    printf("Optimized approach: %.3f ms, result = %.6f\n", optimized_time, optimized_result);
    printf("Speedup: %.2fx\n", naive_time / optimized_time);
    
    return 0;
}

Overwriting codes/performance_comparison.cu


In [19]:
%%bash                                           
nvcc --extended-lambda -o codes/performance_comparison codes/performance_comparison.cu               
./codes/performance_comparison                                                                

Naive approach: 0.657 ms, result = 0.539685
Optimized approach: 0.329 ms, result = 0.539685
Speedup: 2.00x


# Summary
In this notebook, we've explored how to use iterators to extend and optimize standard algorithms. The key takeaways are:

- **Iterators are powerful** abstractions that allow us to work with sequences of elements
- **Transform iterators** apply operations on-the-fly during element access
- **Zip iterators** combine multiple sequences into a single sequence of tuples
- **Combining iterators** can eliminate the need for temporary storage
- **Memory-bound operations** benefit significantly from reducing memory operations

By using these techniques, we optimized our particle simulation to find the maximum displacement without allocating any temporary storage, resulting in significant performance improvements for large simulations.

## Next Steps
You can continue to explore other Thrust iterators like:

- thrust::constant_iterator: Produces a sequence of identical values
- thrust::counting_iterator: Produces a sequence of consecutive values
- thrust::permutation_iterator: Reorders elements of another sequence

These iterators can help you optimize other aspects of particle simulations, like computing physical interactions efficiently.